# 1.导包

In [11]:
import os
import gc
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from PIL import Image
from scipy import signal
import torch.optim as optim
from scipy.io import loadmat
from scipy.ndimage import zoom
import matplotlib.pyplot as plt
import torchvision.models as models
from pyts.image import GramianAngularField
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from sklearn.preprocessing import LabelEncoder
from torch.optim.lr_scheduler import OneCycleLR
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from torchvision.models import resnet50, ResNet50_Weights

In [12]:
# 检查是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
TARGET_IMAGE_SIZE = 224

# 定义常量
WINDOW_SIZE = 512  # 减小窗口大小以减少内存使用
OVERLAP_RATE = 0.4  # 增加步长以减少生成的图像数量
STRIDE = int(WINDOW_SIZE * (1 - OVERLAP_RATE))
MAX_IMAGES_PER_COLUMN = 500 # 每列最多生成的图像数量
print("Seccessful!")

cuda
Seccessful!


# 2.数据预处理
## （1）数据转CSV格式

In [13]:
file_names = ['97.mat', '105.mat', '118.mat', '130.mat', '169.mat',
              '185.mat', '197.mat', '209.mat', '222.mat', '234.mat']
# 采用驱动端数据
data_columns = ['X097_DE_time', 'X105_DE_time', 'X118_DE_time', 'X130_DE_time', 'X169_DE_time',
                'X185_DE_time','X197_DE_time','X209_DE_time','X222_DE_time','X234_DE_time']
columns_name = ['de_normal','de_7_inner','de_7_ball','de_7_outer','de_14_inner','de_14_ball','de_14_outer','de_21_inner','de_21_ball','de_21_outer']
data_12k_1797_10c = pd.DataFrame()
for index in range(10):
    # 读取MAT文件
    data = loadmat(f'../data_deal/{file_names[index]}')
    dataList = data[data_columns[index]].reshape(-1)
    data_12k_1797_10c[columns_name[index]] = dataList[:121265]  # 121048  min: 121265
print(data_12k_1797_10c.shape)
# # 转换为CSV格式文件
# data_12k_1797_10c.set_index('de_normal',inplace=True)
# data_12k_1797_10c.to_csv('data_12k_1797_10c.csv')

(121265, 10)


## （2）一维数据转二维图像

In [14]:
def _data_to_gaf(window_normalized, method='summation'):
    """
    使用 Gramian Angular Field (GAF) 将一维**已归一化**的信号窗口转换为二维矩阵。
    
    注意：此函数期望输入 'window_normalized' 已经被 Min-Max 归一化到 [-1, 1]。

    :param window_normalized: 时域信号窗口 (1D numpy array)，已归一化到 [-1, 1]
    :param method: GAF 类型 ('summation' -> GASF, 'difference' -> GADF)
    :return: GAF 矩阵 (N x N, N=len(window))
    """
    # 移除 'standardization' 参数，使用默认值或指定 'sample_range'
    # GAF 默认使用 [0, pi] 角度范围，要求输入在 [-1, 1] 或 [0, 1] 范围内。
    # 我们将在外部手动归一化到 [-1, 1]。
    transformer = GramianAngularField(
        image_size=len(window_normalized), 
        method=method
        # 移除: standardization='minmax'
        # 可以添加 sample_range=(-1., 1.) 确保角度映射正确
        # sample_range=(-1., 1.) 
    )

    # pyts 要求输入是 (n_samples, n_timestamps) 形状，所以需要 reshape
    X = window_normalized.reshape(1, -1)
    
    # 进行转换
    X_gaf = transformer.fit_transform(X)
    
    # 返回 GAF 矩阵 (形状为 N x N)
    return X_gaf[0]


def generate_gaf_images(column_data, window_size, stride, max_images_per_column, target_size=TARGET_IMAGE_SIZE, gaf_method='summation'):
    """
    使用 Gramian Angular Field (GAF) 生成图像。
    """
    images = []
    image_count = 0
    
    if window_size > len(column_data):
        print("警告: 窗口大小大于数据长度，跳过生成。")
        return images

    for start in range(0, len(column_data) - window_size + 1, stride):
        if image_count >= max_images_per_column:
            break
            
        window = column_data[start:start + window_size].copy()

        # --- 关键修改点 1: 手动 Min-Max 归一化到 [-1, 1] ---
        min_val_window = np.min(window)
        max_val_window = np.max(window)
        
        if max_val_window - min_val_window < 1e-6:
            # 窗口值几乎不变，归一化到 0
            window_normalized = window * 0.0
        else:
            # 归一化到 [-1, 1]：(2 * (x - min) / (max - min)) - 1
            window_normalized = (2 * (window - min_val_window) / (max_val_window - min_val_window)) - 1
        # ----------------------------------------------------

        # 1. 计算 GAF 矩阵 (N x N)
        gaf_matrix = _data_to_gaf(window_normalized, method=gaf_method)
        
        # 2. 归一化到 [0, 1] (将 GAF 值从 [-1, 1] 映射到图像的 [0, 1])
        # 这一步是为了图像显示和后续处理的统一。
        min_gaf = np.min(gaf_matrix)
        max_gaf = np.max(gaf_matrix)
        
        if max_gaf - min_gaf > 1e-6:
            normalized_gaf = (gaf_matrix - min_gaf) / (max_gaf - min_gaf)
        else:
            normalized_gaf = gaf_matrix * 0.0
            
        # 3. 图像缩放/插值到目标尺寸
        h, w = normalized_gaf.shape
        scale = target_size / h 

        downscaled_image = zoom(normalized_gaf, scale, order=3)
        
        # 4. 堆叠成 3 通道 (RGB)
        image = np.stack((downscaled_image,) * 3, axis=-1)
        
        images.append((start, image))
        image_count += 1
        
    return images


# --- GAF 图像处理与保存函数 (保持不变) ---
def process_and_display_column_gaf(data, column_name, window_size, stride, max_images_per_column, target_size=TARGET_IMAGE_SIZE, gaf_method='summation'):
    """ 
    处理并保存指定列的 GAF 图像。
    """
    method_name = "GASF" if gaf_method == 'summation' else "GADF"
    print(f"\n--- 正在使用 {method_name} 处理列: {column_name} ---")
    column_data = data[column_name].values

    # 1. 生成 GAF 图像列表
    # 这里调用修改后的 generate_gaf_images
    gaf_images = generate_gaf_images(column_data, window_size, stride, max_images_per_column, target_size, gaf_method)

    # 创建 'dataset' 文件夹
    dataset_folder = 'dataset'
    if not os.path.exists(dataset_folder):
        os.makedirs(dataset_folder)

    # 创建列名和 GAF 方法对应的文件夹
    column_folder_name = f'{column_name}'
    column_folder = os.path.join(dataset_folder, column_folder_name)
    if not os.path.exists(column_folder):
        os.makedirs(column_folder)

    print(f"生成的图像数量: {len(gaf_images)}")

    # 2. 遍历并保存图像
    for image_index, (start, image) in enumerate(gaf_images):
        plt.figure(figsize=(2.24, 2.24), dpi=100)
        
        plt.imshow(image[:, :, 0], cmap='jet', origin='lower', 
                   vmin=0.0, vmax=1.0) 
        
        plt.title(f'{column_name} {method_name} - Index: {start}', fontsize=8)
        plt.axis('off') 

        image_filename = os.path.join(column_folder, f'{column_name}_{method_name}_{image_index + 1}_S{start}.png')
        plt.savefig(image_filename, bbox_inches='tight', pad_inches=0) 
        plt.close() 

        if (image_index + 1) % 50 == 0:
            gc.collect()

    # 3. 内存清理
    del column_data, gaf_images
    gc.collect()


# --- 运行 GAF 图像生成 (保持不变) ---

# 1. 生成 GASF 图像
for col in data_12k_1797_10c.columns:
    process_and_display_column_gaf(data_12k_1797_10c, col, WINDOW_SIZE, STRIDE, MAX_IMAGES_PER_COLUMN, TARGET_IMAGE_SIZE, gaf_method='summation')

# # 2. 生成 GADF 图像
# for col in data_12k_1797_10c.columns:
#     process_and_display_column_gaf(data_12k_1797_10c, col, WINDOW_SIZE, STRIDE, MAX_IMAGES_PER_COLUMN, TARGET_IMAGE_SIZE, gaf_method='difference')

print("\n所有列的 GAF 图像处理完成。请检查 'dataset' 文件夹以查看生成的图像。")


--- 正在使用 GASF 处理列: de_normal ---
生成的图像数量: 394

--- 正在使用 GASF 处理列: de_7_inner ---
生成的图像数量: 394

--- 正在使用 GASF 处理列: de_7_ball ---
生成的图像数量: 394

--- 正在使用 GASF 处理列: de_7_outer ---
生成的图像数量: 394

--- 正在使用 GASF 处理列: de_14_inner ---
生成的图像数量: 394

--- 正在使用 GASF 处理列: de_14_ball ---
生成的图像数量: 394

--- 正在使用 GASF 处理列: de_14_outer ---
生成的图像数量: 394

--- 正在使用 GASF 处理列: de_21_inner ---
生成的图像数量: 394

--- 正在使用 GASF 处理列: de_21_ball ---
生成的图像数量: 394

--- 正在使用 GASF 处理列: de_21_outer ---
生成的图像数量: 394

所有列的 GAF 图像处理完成。请检查 'dataset' 文件夹以查看生成的图像。


# 3.数据集划分

In [15]:
# 根据需要模型接口调整图像大小
IMAGE_SIZE = (224, 224) 

# 定义数据集路径
dataset_path = 'dataset'

# 初始化数据列表和标签列表
images = []
labels = []

# 遍历数据集文件夹
for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_path):
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            if image_path.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                # 打开并调整图像大小
                img = Image.open(image_path).resize(IMAGE_SIZE, Image.LANCZOS)
                # 将图像转换为numpy数组
                img_array = np.array(img)
                # 添加到图像列表
                images.append(img_array)
                # 添加对应标签
                labels.append(class_name)

# 将图像数据和标签转换为numpy数组
images = np.array(images)
labels = np.array(labels)

# 将类别名称转换为数字标签
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# 打印数据和标签的形状
print(f'Images shape: {images.shape}')
print(f'Labels shape: {labels_encoded.shape}')

# 归一化数据
images = images.astype('float32')
images = images / 255.0

# 划分数据集
X_train, X_temp, y_train, y_temp = train_test_split(images, labels_encoded, test_size=0.3, random_state=42, stratify=labels_encoded)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.333, random_state=42, stratify=y_temp)

# 打印划分后的数据集形状
print(f'Train images shape: {X_train.shape}')
print(f'Train labels shape: {y_train.shape}')
print(f'Validation images shape: {X_val.shape}')
print(f'Validation labels shape: {y_val.shape}')
print(f'Test images shape: {X_test.shape}')
print(f'Test labels shape: {y_test.shape}')

# # 将数据集保存到文件
# np.save('X_train.npy', X_train)
# np.save('y_train.npy', y_train)
# np.save('X_val.npy', X_val)
# np.save('y_val.npy', y_val)
# np.save('X_test.npy', X_test)
# np.save('y_test.npy', y_test)


Images shape: (3940, 224, 224, 4)
Labels shape: (3940,)
Train images shape: (2758, 224, 224, 4)
Train labels shape: (2758,)
Validation images shape: (788, 224, 224, 4)
Validation labels shape: (788,)
Test images shape: (394, 224, 224, 4)
Test labels shape: (394,)


# 4.定义数据加载器，导入模型

In [16]:
# # X_train 和 y_train 已经是 numpy 数组
# X_train = np.load('/kaggle/working/X_train.npy')
# y_train = np.load('/kaggle/working/y_train.npy')
# X_val = np.load('/kaggle/working/X_val.npy')
# y_val = np.load('/kaggle/working/y_val.npy')

# 将 numpy 数组转换为 torch 张量
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).long()
X_val_tensor = torch.from_numpy(X_val).float()
y_val_tensor = torch.from_numpy(y_val).long()

# 创建 TensorDataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

# 定义数据加载器
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print("Data loaded succesfull!")

Data loaded succesfull!


# 5.加载预训练模型，模型微调

In [17]:
# 检查是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载预训练的效果最好的resnet50权重版本
model = resnet50(weights=ResNet50_Weights.DEFAULT)

# 假设你有 10 个类别
num_classes = 10

# 冻结所有参数
for param in model.parameters():
    param.requires_grad = False

for param in model.layer4.parameters():
    param.requires_grad = True

# 获取 ResNet50 最后一个全连接层 (model.fc) 的输入特征数
# 对于 ResNet50，这个值固定是 2048
num_ftrs = model.fc.in_features 

# 修改分类层 (model.fc) 以适应你的分类任务
# 这里使用你提供的更复杂的分类器结构作为示例
model.fc = nn.Sequential(
    # 第一层从 ResNet50 的输出 (2048) 开始
    nn.Linear(num_ftrs, 2048),
    nn.ReLU(),
    nn.Dropout(0.4),
    
    # 输出层：连接到你的类别数
    nn.Linear(2048 , num_classes)
)

# 将模型移动到GPU（如果可用）
model.to(device)

# 打印模型结构
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [18]:
# 定义损失函数
criterion = nn.CrossEntropyLoss()
# 自定义优化器和学习率调度器
LR_CUSTOM_HEAD = 1e-4  # 0.0001 (自定义隐藏层)
LR_LAYER4 = 1e-5       # 0.00001 (解冻的 layer4)
param_groups = [
    # 参数组 1: 自定义 FC Head
    {'params': model.fc.parameters(), 'lr': LR_CUSTOM_HEAD, 'group_name': 'custom_head'},
    
    # 参数组 2: 解冻的 layer4
    {'params': model.layer4.parameters(), 'lr': LR_LAYER4, 'group_name': 'layer4_fine_tune'}
]
optimizer = optim.Adam(param_groups)

# 定义总训练周期和每个周期的批次数量
num_epochs = 200
steps_per_epoch = len(train_loader) 

# OneCycleLR 的 max_lr 参数现在是一个列表，对应优化器中每个参数组的最大学习率
scheduler = OneCycleLR(
    optimizer, 
    max_lr=[LR_CUSTOM_HEAD, LR_LAYER4],
    steps_per_epoch=steps_per_epoch, 
    epochs=num_epochs
)
print("Successful!")

Successful!


# 6.模型训练和验证

In [19]:
# --- 早停策略配置 ---
patience = 40 # 连续10个epoch验证损失没有改善就停止
best_val_loss = float('inf') # 初始最佳验证损失设置为无穷大
BEST_LOSS_THRESHOLD = 0.01 # 新增的停止阈值
epochs_no_improve = 0 # 记录没有改善的epoch数量

# 训练模型
for epoch in range(num_epochs):
    # 训练阶段
    model.train()
    train_losses = []
    train_accuracies = []
    running_loss = 0.0
    correct = 0
    total = 0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.permute(0, 3, 1, 2)
        inputs = inputs[:, :3, :, :]
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    train_accuracy = correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(train_accuracy)
    
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')

    # 验证阶段
    model.eval()
    val_loss = 0.0
    val_losses = []
    val_accuracies = []
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.permute(0, 3, 1, 2)
            inputs = inputs[:, :3, :, :]
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_accuracy = correct / total
    val_epoch_loss = val_loss / len(val_loader)
    val_losses.append(val_epoch_loss)
    val_accuracies.append(val_accuracy)
    
    print(f'Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    # 策略 1: 检查是否达到预设的最低损失阈值
    if val_epoch_loss <= BEST_LOSS_THRESHOLD:
        print(f"\n✨ 验证损失 {val_epoch_loss:.4f} 达到或低于阈值 {BEST_LOSS_THRESHOLD}，停止训练。")
        torch.save(model.state_dict(), "best_resnet50_model.pth")
        print("模型已保存。")
        break
        
    # 策略 2: 标准的基于 patience 的早停
    if val_epoch_loss < best_val_loss:
        best_val_loss = val_epoch_loss
        epochs_no_improve = 0
        # 保存最佳模型
        torch.save(model.state_dict(), "best_resnet50_model.pth")
        print("验证损失降低，保存最佳模型。")
    else:
        epochs_no_improve += 1
        print(f"验证损失未改善，耐心计数: {epochs_no_improve}/{patience}")

    if epochs_no_improve >= patience:
        print(f"\n连续 {patience} 个epoch验证损失没有改善，停止训练。")
        break

Epoch 1/200, Train Loss: 2.2770, Train Accuracy: 0.1682
Validation Loss: 2.2477, Validation Accuracy: 0.3832
验证损失降低，保存最佳模型。
Epoch 2/200, Train Loss: 2.2189, Train Accuracy: 0.3778
Validation Loss: 2.1897, Validation Accuracy: 0.6015
验证损失降低，保存最佳模型。
Epoch 3/200, Train Loss: 2.1547, Train Accuracy: 0.5395
Validation Loss: 2.1247, Validation Accuracy: 0.7183
验证损失降低，保存最佳模型。
Epoch 4/200, Train Loss: 2.0830, Train Accuracy: 0.6095
Validation Loss: 2.0388, Validation Accuracy: 0.7348
验证损失降低，保存最佳模型。
Epoch 5/200, Train Loss: 1.9812, Train Accuracy: 0.6831
Validation Loss: 1.9344, Validation Accuracy: 0.7805
验证损失降低，保存最佳模型。
Epoch 6/200, Train Loss: 1.8606, Train Accuracy: 0.7259
Validation Loss: 1.7973, Validation Accuracy: 0.7868
验证损失降低，保存最佳模型。
Epoch 7/200, Train Loss: 1.7116, Train Accuracy: 0.7389
Validation Loss: 1.6379, Validation Accuracy: 0.8147
验证损失降低，保存最佳模型。
Epoch 8/200, Train Loss: 1.5268, Train Accuracy: 0.7788
Validation Loss: 1.4354, Validation Accuracy: 0.8261
验证损失降低，保存最佳模型。
Epoch 9/

# 7.模型测试

In [20]:
# # 加载 numpy 数组
# X_test = np.load('/kaggle/working/X_test.npy')
# y_test = np.load('/kaggle/working/y_test.npy')

# 将 numpy 数组转换为 torch 张量
# X_test 转换为 float 类型以匹配模型输入
X_test_tensor = torch.from_numpy(X_test).float()
# y_test 转换为 long 类型以匹配 CrossEntropyLoss 的期望
y_test_tensor = torch.from_numpy(y_test).long()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 创建 TensorDataset
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# 定义数据加载器
batch_size = 32
# 测试集通常不需要打乱，因此 shuffle=False
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

MODEL_PATH = "best_resnet50_model.pth"
print(f"\n正在加载模型文件：{MODEL_PATH}")
# 使用 map_location 将模型加载到正确的设备上
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
print("模型加载完成。")

print("\n开始进行最终测试...")
model.eval()
all_labels = []
all_preds = []
test_loss = 0.0
with torch.no_grad():
    for inputs, labels in test_loader:
        # 数据预处理
        inputs = inputs.permute(0, 3, 1, 2)
        inputs = inputs[:, :3, :, :]
        inputs, labels = inputs.to(device), labels.to(device)
        
        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        
        # 收集所有真实标签和预测标签
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

test_accuracy = np.mean(np.array(all_labels) == np.array(all_preds))
test_epoch_loss = test_loss / len(test_loader)
print(f'Test Loss: {test_epoch_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')


正在加载模型文件：best_resnet50_model.pth
模型加载完成。

开始进行最终测试...
Test Loss: 0.0114, Test Accuracy: 1.0000
